# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader) # skip header
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


['Stephen Lynch', 'Logged In', 'Jayden', 'M', '0', 'Bell', '182.85669', 'free', 'Dallas-Fort Worth-Arlington, TX', 'PUT', 'NextSong', '1.54099E+12', '829', "Jim Henson's Dead", '200', '1.54354E+12', '91']


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    create keyspace if not exists udacity
    with replication = 
    {'class': 'SimpleStrategy', 'replication_factor': 1 }"""
    )
    
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query #1:
### Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession = 4
To answer this question, we need to select artist, song title, song length from our table and we need to filter using sessionId and itemInSession. 

CQL will look like this:
`SELECT artist, song_title, song_length FROM session_songs WHERE sessionId = 338 and itemInSession = 4`

- We will name our table **session_songs**
- Our primary key will consist of partition key sessionId, and clustering key itemInSession so that we can filter by this attributes later on.
- The columns of our table will be: sessionId, itemInSession, artist, song_title and song_length.

#### Create section

In [8]:
drop_query = "DROP TABLE IF EXISTS session_songs"
query = "CREATE TABLE IF NOT EXISTS session_songs "
query = query + "(sessionId int, itemInSession int, artist text, song_title text, song_length float, PRIMARY KEY (sessionId, ItemInSession))"

try:
    session.execute(drop_query)
    session.execute(query)
except Exception as e:
    print(e)
                    

#### Insert section

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session_songs (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        artist, user_name, gender, itemInSession, user_last_name, song_length, level, location, sessionId, song_title, userId = line
        session.execute(query, (int(sessionId), int(itemInSession), artist, song_title, float(song_length)))

#### Select section

In [10]:
query = "select artist, song_title, song_length from session_songs WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Query #2:
### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
To answer this question, we need to select itemInSession, artist, song title, first name, last name from our table and we need to filter using userid and sessionId. 

CQL will look like this:
`SELECT itemInSession, artist, song_title, firstName, lastName FROM user_songs WHERE userId = 10 and sessionId = 182`

- We will name our table **user_songs**
- Our primary key will consist of composite partition key userId and sessionId. The reason for this is that if we only use userId as partition key, the sessionid which belongs to the same user will be put into different nodes since userId is unique in each node. Clustering key should be itemInSession so that we can filter by it later on.
- The columns of our table will be: userId, sessionId, itemInSession, artist, song_title, firstName and lastName.

#### Create section

In [11]:
drop_query = "DROP TABLE IF EXISTS user_songs"
query = "CREATE TABLE IF NOT EXISTS user_songs "
query = query + "(userId int, sessionId int, ItemInSession int, artist text, song_title text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), ItemInSession))"

try:
    session.execute(drop_query)
    session.execute(query)
except Exception as e:
    print(e)        

#### Insert section

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_songs (userId, sessionId, itemInSession, artist, song_title, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song_title, userId = line
        session.execute(query, (int(userId), int(sessionId), int(itemInSession), artist, song_title, firstName, lastName))
 

#### Select section

In [13]:
query = "select itemInSession, artist, song_title, firstName, lastName from user_songs WHERE userId = 10 and sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    #print(row)
    print(row.iteminsession, row.artist, row.song_title, row.firstname, row.lastname)

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query #3:
### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
To answer this question, we need to select user first name, last name from our table and we need to filter using song title. As the user name is not unique in the table, we need to add userId in the PRIMARY KEY.

CQL will look like this:
`SELECT firstName, lastName FROM app_history WHERE song_title = 'All Hands Against His Own'`

- We will name our table **app_history**
- Our primary key will consist of partition key song_title and clustering column userId.
- The columns of our table will be: song_title, userId, firstName and lastName.

#### Create section

In [14]:
drop_query = "DROP TABLE IF EXISTS app_history"
query = "CREATE TABLE IF NOT EXISTS app_history "
query = query + "(song_title text, userId int, firstName text, lastName text, PRIMARY KEY (song_title, userId))"

try:
    session.execute(drop_query)
    session.execute(query)
except Exception as e:
    print(e)        
                    

#### Insert section

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO app_history (song_title, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        # artist, firstName, gender, ItemInSession, lastName, length, level, location, sessionId, title, userId = line
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
 

#### Select section

In [16]:
query = "select firstName, lastName from app_history WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    #print(row)
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
session.execute("DROP TABLE session_songs")
session.execute("DROP TABLE user_songs")
session.execute("DROP TABLE app_history")

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()